In [3]:
# --- 0) 경로 변수 먼저 정의 ---
import os, glob, shutil
from pathlib import Path
from sklearn.model_selection import train_test_split

BASE = '/content/drive/MyDrive/인공지능 튜플'   # <- 구글 드라이브 루트(마운트 필요)
DATA_DIR = os.path.join(BASE, 'OriginalDataset')  # <- 원본(클래스 폴더만 있는 단일 루트)
SPLIT = '/content/dataset_hybrid'                 # <- 분할본을 만들 위치(로컬)

assert os.path.isdir(DATA_DIR), f"원본 데이터 폴더가 없습니다: {DATA_DIR}\n드라이브 마운트/경로를 확인하세요."

# --- 1) train/val/test 물리 분할 생성(없으면 생성, 있으면 스킵) ---
R_TRAIN, R_VAL, R_TEST = 0.8, 0.1, 0.1
SEED = 123
EXTS = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')

if not (os.path.isdir(os.path.join(SPLIT, 'train')) and
        os.path.isdir(os.path.join(SPLIT, 'val')) and
        os.path.isdir(os.path.join(SPLIT, 'test'))):
    classes = [d for d in sorted(os.listdir(DATA_DIR)) if os.path.isdir(os.path.join(DATA_DIR, d))]
    print("클래스:", classes)
    for split in ['train','val','test']:
        for c in classes:
            os.makedirs(os.path.join(SPLIT, split, c), exist_ok=True)

    for c in classes:
        files = []
        for e in EXTS:
            files += glob.glob(os.path.join(DATA_DIR, c, f'*{e}'))
        files = sorted(files)
        if len(files) == 0:
            print(f"[warn] '{c}' 클래스에 이미지가 없습니다.")
            continue

        train_files, rest = train_test_split(files, test_size=(1 - R_TRAIN), random_state=SEED, shuffle=True)
        val_files, test_files = train_test_split(rest, test_size=R_TEST/(R_VAL+R_TEST), random_state=SEED, shuffle=True)

        for src_list, dst_split in [(train_files,'train'), (val_files,'val'), (test_files,'test')]:
            dst_dir = os.path.join(SPLIT, dst_split, c)
            for src in src_list:
                shutil.copy2(src, os.path.join(dst_dir, os.path.basename(src)))
    print(f"[ok] 분할 생성 완료: {SPLIT}")
else:
    print(f"[skip] 이미 분할 폴더가 존재합니다: {SPLIT}")


클래스: ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
[ok] 분할 생성 완료: /content/dataset_hybrid


In [6]:
# ============================================================
# CNN–ViT Hybrid (ResNet50 -> ViT) | Robust CKPT + Masked Rollout + TTA
# - CKPT 우선순위/형식 감지 로더: *.weights.h5 > .keras(zip) > .h5
# - Validation/Test 로더 (증강 X, [0,1] 스케일)
# - Attention Rollout (두상 마스크 내부 정규화 + 배경 알파=0)
# - gt|pred 타이틀 그리드 저장 + 진단 지표(brain-focus ratio)
# - TTA 정책별 성능 비교 (CSV/Confusion Matrix 저장)
# 저장 경로: /content/drive/MyDrive/인공지능 튜플/proj_pr/out/
# ============================================================
import os, glob, time, zipfile, csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score

# -------------------
# 0) 경로/상수
# -------------------
from google.colab import drive
drive.mount('/content/drive')

BASE  = '/content/drive/MyDrive/인공지능 튜플'
SPLIT = '/content/dataset_hybrid'   # val/ , test/ 가 있는 경로
OUT   = f'{BASE}/proj_pr/out'
os.makedirs(OUT, exist_ok=True)

IMG_SIZE    = (288, 288)
BATCH       = 32
SEED        = 123
NUM_CLASSES = 4

tf.keras.utils.set_random_seed(SEED)

# -------------------
# 1) 모델 정의 (직렬화 가능)
# -------------------
class PositionalEmbedding2D(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.pos = None
    def build(self, input_shape):
        H, W, D = input_shape[1], input_shape[2], input_shape[3]
        if None in (H, W, D):
            raise ValueError(f"PositionalEmbedding2D needs static H,W,D, got {input_shape}")
        self.pos = self.add_weight(
            name="pos2d",
            shape=(H, W, D),
            initializer=keras.initializers.RandomNormal(stddev=0.02),
            trainable=True,
        )
    def call(self, x):  # x: (B,H,W,D)
        return x + self.pos
    def get_config(self):
        return super().get_config()

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads=4, mlp_ratio=4.0, drop=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.mlp_ratio = mlp_ratio
        self.drop = drop
        self.n1 = layers.LayerNormalization(epsilon=1e-6)
        self.mha= layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim//num_heads, dropout=drop)
        self.d1 = layers.Dropout(drop)
        self.n2 = layers.LayerNormalization(epsilon=1e-6)
        self.mlp= keras.Sequential([
            layers.Dense(int(embed_dim*mlp_ratio), activation=tf.nn.gelu),
            layers.Dropout(drop),
            layers.Dense(embed_dim),
            layers.Dropout(drop),
        ])
    def call(self, x, training=False):
        h = self.mha(self.n1(x), self.n1(x), training=training)
        x = x + self.d1(h, training=training)
        x = x + self.mlp(self.n2(x), training=training)
        return x
    def get_config(self):
        cfg = super().get_config()
        cfg.update(dict(embed_dim=self.embed_dim, num_heads=self.num_heads,
                        mlp_ratio=self.mlp_ratio, drop=self.drop))
        return cfg

def build_hybrid(input_shape, num_classes, embed_dim=256, vit_blocks=4, heads=4, drop=0.1):
    inp = layers.Input(input_shape)
    x = keras.applications.resnet.preprocess_input(inp*255.0)
    backbone = keras.applications.ResNet50(include_top=False, weights='imagenet', name='resnet50')
    feat = backbone(x, training=False)

    tok = layers.Conv2D(embed_dim, 2, strides=2, padding='same', name='patch_embed_conv')(feat)
    tok = layers.BatchNormalization(name='patch_embed_bn')(tok)
    tok = layers.Activation('gelu', name='patch_embed_gelu')(tok)
    tok = layers.Dropout(drop, name='patch_embed_dropout')(tok)

    tok = PositionalEmbedding2D(name='pos2d')(tok)
    H, W = tok.shape[1], tok.shape[2]
    tokens = layers.Reshape((H*W, embed_dim), name='tokens')(tok)
    z = layers.Dropout(drop, name='tokens_dropout')(tokens)

    for i in range(vit_blocks):
        z = TransformerEncoder(embed_dim=embed_dim, num_heads=heads, mlp_ratio=4.0, drop=drop, name=f'encoder_{i}')(z)

    z = layers.GlobalAveragePooling1D(name='gap')(z)
    z = layers.Dropout(0.5, name='head_drop1')(z)
    z = layers.Dense(512, activation='gelu', name='head_dense')(z)
    z = layers.Dropout(0.4, name='head_drop2')(z)
    out = layers.Dense(num_classes, activation='softmax', name='classifier')(z)

    model = keras.Model(inp, out, name='ResNet50_ViT_hybrid')
    for l in backbone.layers:
        if isinstance(l, layers.BatchNormalization): l.trainable = False
    return model

# -------------------
# 2) CKPT 선택/로딩 (우선순위 + 형식 감지)
# -------------------
def pick_ckpt(out_dir):
    # 1순위: *.weights.h5  2순위: .keras(zip)  3순위: .h5
    w = sorted(glob.glob(os.path.join(out_dir, '*.weights.h5')), key=os.path.getmtime, reverse=True)
    k = sorted(glob.glob(os.path.join(out_dir, '*.keras')), key=os.path.getmtime, reverse=True)
    h = sorted([p for p in glob.glob(os.path.join(out_dir, '*.h5')) if not p.endswith('.weights.h5')], key=os.path.getmtime, reverse=True)
    cand = (w + k + h)
    print("[CKPT candidates]")
    for p in cand[:8]:
        sz = os.path.getsize(p) if os.path.exists(p) else -1
        print(" -", os.path.basename(p), f"({sz} bytes)")
    return cand[0] if cand else None

def is_valid_keras_zip(path):  # .keras가 진짜 zip인지 확인
    try:
        return zipfile.is_zipfile(path)
    except:
        return False

CKPT = pick_ckpt(OUT)
assert CKPT is not None, f"체크포인트 없음: {OUT}"

roll_model = None
if CKPT.endswith('.weights.h5'):
    print("[i] weights-only HDF5 → build + load_weights")
    tmp = build_hybrid(IMG_SIZE+(3,), NUM_CLASSES, embed_dim=256, vit_blocks=4, heads=4, drop=0.1)
    tmp.load_weights(CKPT)
    roll_model = tmp
    print("[✓] load_weights:", os.path.basename(CKPT))

elif CKPT.endswith('.keras'):
    if is_valid_keras_zip(CKPT):
        print("[i] valid .keras zip → load_model")
        roll_model = keras.models.load_model(
            CKPT, compile=False,
            custom_objects={'PositionalEmbedding2D': PositionalEmbedding2D,
                            'TransformerEncoder': TransformerEncoder}
        )
        print("[✓] load_model(.keras):", os.path.basename(CKPT))
    else:
        print("[!] .keras가 zip 포맷이 아님 → weights로 시도")
        tmp = build_hybrid(IMG_SIZE+(3,), NUM_CLASSES, embed_dim=256, vit_blocks=4, heads=4, drop=0.1)
        tmp.load_weights(CKPT)
        roll_model = tmp
        print("[✓] load_weights(.keras명):", os.path.basename(CKPT))
else:  # 기타 .h5
    print("[i] .h5 감지 → load_model 시도, 실패 시 weights로 재시도")
    try:
        roll_model = keras.models.load_model(
            CKPT, compile=False,
            custom_objects={'PositionalEmbedding2D': PositionalEmbedding2D,
                            'TransformerEncoder': TransformerEncoder}
        )
        print("[✓] load_model(.h5):", os.path.basename(CKPT))
    except Exception as e:
        print("[!] load_model 실패 → load_weights:", e)
        tmp = build_hybrid(IMG_SIZE+(3,), NUM_CLASSES, embed_dim=256, vit_blocks=4, heads=4, drop=0.1)
        tmp.load_weights(CKPT)
        roll_model = tmp
        print("[✓] load_weights(.h5):", os.path.basename(CKPT))

# -------------------
# 3) Val/Test 로더 (증강 X, [0,1])
# -------------------
val_raw = tf.keras.utils.image_dataset_from_directory(
    os.path.join(SPLIT, 'val'), image_size=IMG_SIZE, batch_size=BATCH, shuffle=False)
test_raw = tf.keras.utils.image_dataset_from_directory(
    os.path.join(SPLIT, 'test'), image_size=IMG_SIZE, batch_size=BATCH, shuffle=False)

classes = val_raw.class_names
val_ds  = val_raw.map(lambda x,y: (x/255.0, tf.one_hot(y, NUM_CLASSES))).prefetch(tf.data.AUTOTUNE)
test_ds = test_raw.map(lambda x,y: (x/255.0, tf.one_hot(y, NUM_CLASSES))).prefetch(tf.data.AUTOTUNE)
print("Classes:", classes)

# -------------------
# 4) Rollout 준비 (레이어 핸들/토큰 추출)
# -------------------
def get_layer_safe(model, name=None, type_=None, start_after=None):
    if name is not None:
        try:
            return model.get_layer(name)
        except:
            pass
    if type_ is not None:
        layers_ = model.layers
        if start_after is not None:
            layers_ = layers_[model.layers.index(start_after)+1:]
        for l in layers_:
            if isinstance(l, type_): return l
    raise ValueError(f"Layer not found: name={name}, type={type_}")

L = roll_model.layers
resnet = get_layer_safe(roll_model, name='resnet50')
conv_pe = get_layer_safe(roll_model, name='patch_embed_conv', type_=layers.Conv2D, start_after=resnet)
bn_pe   = get_layer_safe(roll_model, name='patch_embed_bn',   type_=layers.BatchNormalization)
gelu_pe = get_layer_safe(roll_model, name='patch_embed_gelu')
drop0   = get_layer_safe(roll_model, name='patch_embed_dropout', type_=layers.Dropout)
pos2d   = get_layer_safe(roll_model, name='pos2d', type_=PositionalEmbedding2D)
reshape = get_layer_safe(roll_model, name='tokens', type_=layers.Reshape)
drop_tk = get_layer_safe(roll_model, name='tokens_dropout', type_=layers.Dropout)
encoders = [l for l in L if isinstance(l, TransformerEncoder)]
assert encoders, "TransformerEncoder 레이어를 찾지 못했습니다."

@tf.function(jit_compile=False)
def tokens_from_image(x):  # x: [0,1]
    x = keras.applications.resnet.preprocess_input(x*255.0)
    f = resnet(x, training=False)
    t = conv_pe(f, training=False)
    t = bn_pe(t, training=False)
    t = gelu_pe(t)
    t = drop0(t, training=False)
    t = pos2d(t, training=False)
    H = tf.shape(t)[1]; W = tf.shape(t)[2]
    z = reshape(t); z = drop_tk(z, training=False)  # (B,T,D)
    return z, H, W

# --- 안전 assert ---
def _assert_row_stochastic(A, atol=1e-4):  # A: (B,Tq,Tk)
    row_sum = tf.reduce_sum(A, axis=-1)
    tf.debugging.assert_near(row_sum, tf.ones_like(row_sum), atol=atol,
                             message="Attention rows must sum to 1 (row-stochastic).")

def _assert_grid_tokens(H, W, T):
    tf.debugging.assert_equal(tf.math.multiply(H, W), T,
                              message="Grid H*W must equal #tokens (reshape mismatch).")

# -------------------
# 5) 두상 마스크 + 마스크 내부 정규화 롤아웃
# -------------------
def brain_mask_smart(x, rel=0.20, min_thr=0.05, morph=2):
    """x: [0,1] NHWC → 상대 임계 + 모폴로지 닫기로 두상 마스크"""
    g = tf.image.rgb_to_grayscale(x)                         # (B,H,W,1)
    vmax = tf.reduce_max(g, axis=[1,2,3], keepdims=True)
    thr  = tf.maximum(min_thr, rel * vmax)
    m = tf.cast(g > thr, tf.float32)                         # 0/1

    k = 3
    for _ in range(morph):                                   # dilate
        m = tf.nn.max_pool2d(m, ksize=k, strides=1, padding='SAME')
    for _ in range(morph):                                   # erode
        m = 1.0 - tf.nn.max_pool2d(1.0 - m, ksize=k, strides=1, padding='SAME')
    return m                                                 # (B,H,W,1)

def compute_rollout_map_masked(
    images, head_fuse='mean', use_residual=True,
    mask_rel=0.20, mask_min_thr=0.05, gamma=0.75
):
    images_tf = tf.convert_to_tensor(images, dtype=tf.float32) if isinstance(images, np.ndarray) else images
    z, H, W = tokens_from_image(images_tf)
    B = tf.shape(z)[0]; T = tf.shape(z)[1]
    _assert_grid_tokens(H, W, T)

    # Rollout 누적
    R = tf.eye(T, batch_shape=[B])
    cur = z
    for enc in encoders:
        q = enc.n1(cur)
        attn_out, scores = enc.mha(q, q, return_attention_scores=True, training=False)  # (B, heads, T, T)

        # 1) 수치 안정화 + 확률화(로짓이어도 OK)
        scores = tf.where(tf.math.is_finite(scores), scores, tf.zeros_like(scores))
        scores = tf.nn.softmax(scores, axis=-1)

        # 2) head 융합
        if head_fuse == 'max':
            A = tf.reduce_max(scores, axis=1)     # (B, T, T)
        else:
            A = tf.reduce_mean(scores, axis=1)    # (B, T, T)

        # 3) 융합 직후 행 정규화
        A = A / (tf.reduce_sum(A, axis=-1, keepdims=True) + 1e-6)

        # 4) residual 추가 후 재정규화
        if use_residual:
            A = A + tf.eye(T, batch_shape=[B])
            A = A / (tf.reduce_sum(A, axis=-1, keepdims=True) + 1e-6)

        # 5) 누적
        R = tf.linalg.matmul(R, A)

        # 6) 트랜스포머 전진
        cur = cur + enc.d1(attn_out, training=False)
        h = enc.n2(cur); h = enc.mlp(h, training=False)
        cur = cur + h

    # 토큰 중요도 → (B, H, W, 1) → 원 해상도 보간
    heat = tf.reduce_mean(R, axis=1)                          # (B,T)
    heat = tf.reshape(heat, (-1, H, W, 1))
    heat = tf.image.resize(heat, tf.shape(images_tf)[1:3], method='bilinear')

    # 마스크 내부 기준 정규화
    m = brain_mask_smart(images_tf, rel=mask_rel, min_thr=mask_min_thr)      # (B,H,W,1)
    heat = heat * m
    very_neg = tf.constant(-1e9, heat.dtype)
    very_pos = tf.constant(1e9,  heat.dtype)
    hmax = tf.reduce_max(tf.where(m>0, heat, very_neg), axis=[1,2,3], keepdims=True)
    hmin = tf.reduce_min(tf.where(m>0, heat, very_pos), axis=[1,2,3], keepdims=True)
    heat = tf.clip_by_value((heat - hmin) / (hmax - hmin + 1e-8), 0., 1.)
    heat = tf.pow(heat, gamma)                                 # 스팟 강조

    return heat.numpy()[..., 0], m.numpy()[..., 0]             # (B,H,W), (B,H,W)


def save_rollout_grid_masked(ds, mdl, classes, save_path, n=12, cols=6,
                             head_fuse='mean', mask_rel=0.20, mask_min_thr=0.05, gamma=0.75):
    # 샘플 뽑기
    xs, ys = [], []
    for x, y in ds.unbatch().take(n):
        xs.append(x.numpy()); ys.append(np.argmax(y.numpy()))
    xs = np.stack(xs, axis=0); ys = np.array(ys)

    # 예측
    probs = mdl.predict(xs, verbose=0)
    preds = np.argmax(probs, axis=1)

    # 히트맵 + 마스크
    heat, mask = compute_rollout_map_masked(xs, head_fuse=head_fuse,
                                            mask_rel=mask_rel, mask_min_thr=mask_min_thr, gamma=gamma)

    # 그리드 렌더링 (배경 알파=0)
    rows = int(np.ceil(n/cols))
    fig, axes = plt.subplots(rows, cols, figsize=(3.4*cols, 3.6*rows))
    axes = np.array(axes).reshape(rows, cols)
    k = 0
    for r in range(rows):
        for c in range(cols):
            ax = axes[r, c]; ax.axis('off')
            if k < n:
                ax.imshow(xs[k])
                ax.imshow(heat[k], cmap='jet', alpha=0.45*mask[k])  # ★ 배경 투명
                gt = classes[ys[k]]; pd = classes[preds[k]]
                ax.set_title(f"gt={gt} | pred={pd}", fontsize=11)
                k += 1
    plt.tight_layout()
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, dpi=200); plt.close()

def rollout_diagnostics(ds, mdl, head_fuse='mean', n=48,
                        mask_rel=0.20, mask_min_thr=0.05, gamma=0.75):
    xs = np.stack([x.numpy() for x,_ in ds.unbatch().take(n)], axis=0)
    heat, mask = compute_rollout_map_masked(xs, head_fuse=head_fuse,
                                            mask_rel=mask_rel, mask_min_thr=mask_min_thr, gamma=gamma)
    inside = float((heat*mask).sum())
    total  = float(heat.sum() + 1e-8)
    brain_focus = inside/total
    # 토큰 그리드 진단
    z,H,W = tokens_from_image(tf.convert_to_tensor(xs, tf.float32))
    T = int(z.shape[1])
    print(f"[diag] grid HxW={int(H.numpy())}x{int(W.numpy())}, T={T}, ok={int(H.numpy())*int(W.numpy())==T}")
    print(f"[diag] brain-focus ratio={brain_focus:.3f} (권장 ≥ 0.90)")
    return brain_focus

# -------------------
# 6) 예시: 그리드/진단 실행
# -------------------
grid_val = f"{OUT}/rollout/val_rollout_grid_masked.png"
save_rollout_grid_masked(val_ds, roll_model, classes, grid_val,
                         n=12, cols=6, head_fuse='max', mask_rel=0.20, mask_min_thr=0.05, gamma=0.75)
print("Saved:", grid_val)

# 원하면 test도
grid_test = f"{OUT}/rollout/test_rollout_grid_masked.png"
save_rollout_grid_masked(test_ds, roll_model, classes, grid_test,
                         n=12, cols=6, head_fuse='max', mask_rel=0.20, mask_min_thr=0.05, gamma=0.75)
print("Saved:", grid_test)

# 수치 진단
rollout_diagnostics(val_ds, roll_model, head_fuse='max', n=48,
                    mask_rel=0.20, mask_min_thr=0.05, gamma=0.75)

# -------------------
# 7) TTA 성능 비교 (Test)
# -------------------
def five_crops(x, pad=16):
    H, W = x.shape[1], x.shape[2]
    x_pad = tf.pad(x, [[0,0],[pad,pad],[pad,pad],[0,0]], mode='REFLECT')
    crops = []
    crops.append(tf.image.crop_to_bounding_box(x_pad, 0, 0,     H, W))       # TL
    crops.append(tf.image.crop_to_bounding_box(x_pad, 0, 2*pad, H, W))       # TR
    crops.append(tf.image.crop_to_bounding_box(x_pad, 2*pad, 0, H, W))       # BL
    crops.append(tf.image.crop_to_bounding_box(x_pad, 2*pad, 2*pad, H, W))   # BR
    crops.append(tf.image.crop_to_bounding_box(x_pad, pad, pad, H, W))       # Center
    return crops

def predict_tta(batch_x, policy, mdl):
    if policy == 'none':
        views = [batch_x]
    elif policy == 'hflip':
        views = [batch_x, tf.image.flip_left_right(batch_x)]
    elif policy == '5crop':
        views = five_crops(batch_x)
    elif policy == 'hflip+5crop':
        base = five_crops(batch_x); views = base + [tf.image.flip_left_right(c) for c in base]
    else:
        raise ValueError("unknown TTA policy")
    probs = [mdl.predict(v, verbose=0) for v in views]
    return np.stack(probs, axis=0).mean(axis=0)

def eval_ds_tta(ds, classes, policy, mdl):
    y_true, y_pred, probs = [], [], []
    n_img = 0; t0 = time.time()
    for x, y in ds:
        p = predict_tta(x, policy, mdl)
        probs.append(p)
        y_true.extend(np.argmax(y.numpy(), axis=1))
        y_pred.extend(np.argmax(p, axis=1))
        n_img += x.shape[0]
    dt = time.time() - t0

    probs = np.concatenate(probs, 0)
    y_true = np.array(y_true); y_pred = np.array(y_pred)
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro')
    auc = roc_auc_score(y_true, probs, multi_class='ovr', average='macro')
    ms_img = dt / n_img * 1000.0
    cm = confusion_matrix(y_true, y_pred)

    print(f"[TTA:{policy:11s}] acc={acc:.4f}  macroF1={mf1:.4f}  ROC-AUC={auc:.4f}  {ms_img:.1f} ms/img")
    return acc, mf1, auc, ms_img, cm

policies = ['none', 'hflip', '5crop', 'hflip+5crop']
rows = []; cms = {}
for pol in policies:
    acc, mf1, auc, ms, cm = eval_ds_tta(test_ds, classes, pol, roll_model)
    rows.append([pol, acc, mf1, auc, ms]); cms[pol] = cm

csv_path = f"{OUT}/tta_results.csv"
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["policy","accuracy","macro_f1","roc_auc","ms_per_image"]); w.writerows(rows)
print("Saved CSV:", csv_path)

for pol, cm in cms.items():
    plt.figure(figsize=(5.6,4.8))
    plt.imshow(cm, cmap='Blues'); plt.title(f'Confusion Matrix (Test, {pol})'); plt.colorbar()
    ticks = np.arange(len(classes))
    plt.xticks(ticks, classes, rotation=45, ha='right'); plt.yticks(ticks, classes)
    for i in range(len(classes)):
        for j in range(len(classes)):
            v=cm[i,j]; c='white' if v>cm.max()/2 else 'black'
            plt.text(j,i,str(v),ha='center',va='center',color=c,fontsize=9)
    plt.tight_layout(); plt.savefig(f"{OUT}/confmat_test_{pol}.png", dpi=220); plt.close()

print("Artifacts saved in:", OUT)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[CKPT candidates]
 - best_hybrid_by_f1.weights.h5 (347782520 bytes)
 - best_hybrid.keras (0 bytes)
 - best_hybrid_by_f1.keras (0 bytes)
 - best_hybrid.h5 (347652312 bytes)
 - best_balanced_focal.h5 (74671448 bytes)
 - best_res288.h5 (75275608 bytes)
[i] weights-only HDF5 → build + load_weights
[✓] load_weights: best_hybrid_by_f1.weights.h5
Found 643 files belonging to 4 classes.
Found 644 files belonging to 4 classes.
Classes: ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
Saved: /content/drive/MyDrive/인공지능 튜플/proj_pr/out/rollout/val_rollout_grid_masked.png
Saved: /content/drive/MyDrive/인공지능 튜플/proj_pr/out/rollout/test_rollout_grid_masked.png
[diag] grid HxW=5x5, T=25, ok=True
[diag] brain-focus ratio=1.000 (권장 ≥ 0.90)
[TTA:none       ] acc=0.5047  macroF1=0.3538  ROC-AUC=0.7859  48.5 ms/img
[TTA:hflip      ] acc=0.5124  macroF1=0.356